In [5]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)


In [6]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

In [7]:

prompt = ChatPromptTemplate(
    [
        
        SystemMessage("Tu eres leomora un assitente de estudios"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

legacy_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
)


/tmp/ipykernel_4284/1114234516.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_4284/1114234516.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  legacy_chain = LLMChain(


In [8]:
legacy_result = legacy_chain.invoke({"text": "my name is bob"})
print(legacy_result)



{'text': "Nice to meet you, Bob! I'm Leomora, your study assistant. How can I help you today? Do you need help with a specific subject or just want to get organized for the week?", 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Bob! I'm Leomora, your study assistant. How can I help you today? Do you need help with a specific subject or just want to get organized for the week?", additional_kwargs={}, response_metadata={})]}


In [9]:
legacy_result = legacy_chain.invoke({"text": "what was my name"})
legacy_result 

{'text': 'Your name is Bob!',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Bob! I'm Leomora, your study assistant. How can I help you today? Do you need help with a specific subject or just want to get organized for the week?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what was my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={})]}

In [10]:
import ollama

In [19]:
emb= ollama.embeddings(model='mxbai-embed-large', prompt='Represent this sentence for searching relevant passages: The sky is blue because of Rayleigh scattering')

In [20]:
len(emb["embedding"])

1024

In [30]:
from pinecone import Pinecone, PodSpec

In [35]:
PINECONE_API_KEY="d078b458-2829-4a0e-adb8-534011e5d430"
pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'lanto-6kvqwt9.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'lanto',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [32]:
pc.delete_index("lanto")

In [34]:
index_name = "lanto"

pc.create_index(
name = index_name,
dimension = 1024,
spec = PodSpec(environment="gcp-starter")
)

In [20]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(
    "El libro de oro.pdf",
)


In [21]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'El libro de oro.pdf', 'page': 0}, page_content=' 1 \n \n \n   \nEL LIBRO DE ORO  \nDE  \n \n \n \n \nSAINT GERMAIN \n \n                  \n \n \n ')

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)


In [23]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="mxbai-embed-large"
)


In [10]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings 
embeddings = HuggingFaceBgeEmbeddings(model_name="all-mpnet-base-v2")

In [24]:
import os
from langchain_pinecone import PineconeVectorStore
os.environ["PINECONE_API_KEY"] =  PINECONE_API_KEY
# Configurar Pinecone para utilizar la GPU
os.environ["PINECONE_GPU"] = "true"

vector_store = PineconeVectorStore(embedding=embed, index_name = index_name)




In [17]:
pc.delete_index("lanto")

/home/leomorya/Especializacion/Materias/Desarrollo_de_proyectos_con_IA/proyecto final/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/leomorya/Especializacion/Materias/Desarrollo_de_proyectos_con_IA/proyecto final/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
vector_store.from_documents(splits, embedding=embed, index_name = index_name)

In [45]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "que es la presencia yo soy?"

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)


In [31]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)




In [32]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different versions of the original question:', 'What are the characteristics or features associated with the concept "presence" that I am?', 'What does it mean to be present, and how is this related to my own existence or identity?', 'How do people or entities typically experience or understand the concept of presence in relation to themselves?']


10

In [ ]:
import pandas as pd

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import OllamaEmbeddings
import pandas as pd
import os
from langchain_pinecone import PineconeVectorStore
import ftfy
PINECONE_API_KEY="d078b458-2829-4a0e-adb8-534011e5d430"
os.environ["PINECONE_API_KEY"] =  PINECONE_API_KEY
index_name = "lanto"


def cargar_libros(path_file, comentario):
    
    loader = PyPDFLoader(path_file)
    docs = loader.load()
    for doc in docs:
        doc.page_content = ftfy.fix_text(doc.page_content)
    df = pd.read_csv("file_libros")
    
    npaginas = len(docs)
    titulo = docs[0].metadata["source"]
    df.loc[len(df)] = [titulo, npaginas]

    
    if df["Título"].duplicated().sum()!=0:
        return "Ya existe este documento en la base de datos vectorial"

      
    df.to_csv("file_libros", index=False)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    splits = text_splitter.split_documents(docs)
    embed = OllamaEmbeddings(model="mxbai-embed-large")
    vector_store = PineconeVectorStore(embedding=embed, index_name = index_name)
    vector_store.from_documents(splits, embedding=embed, index_name = index_name)
    
    return "el archivo path_file a sido cargado con exito"  
    

    

In [3]:
ruta = "El libro de oro.pdf"
cargar_libros(ruta, "este libro es genial")

'Ya existe este documento en la base de datos vectorial'

In [4]:
df = pd.read_csv("file_libros")
df

,Título,"Número de páginas, comentarios"
0,El libro de oro.pdf,74


In [1]:
#import pandas as pd
#df = pd.DataFrame
# Crear DataFrame vacío
#df = pd.DataFrame(columns=["Título", "Número de páginas, Comentarios"])


In [2]:
#df = pd.DataFrame(columns=["Título", "Número de páginas"])
#df.to_csv("file_libros", index=False)

## Multiretriver 

In [7]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama


PINECONE_API_KEY="d078b458-2829-4a0e-adb8-534011e5d430"
os.environ["PINECONE_API_KEY"] =  PINECONE_API_KEY
index_name = "lanto"

embed = OllamaEmbeddings(model="mxbai-embed-large")
vector_store = PineconeVectorStore(embedding=embed, index_name = index_name)

llm = ChatOllama(
    model="llama3.1",
    temperature=0.2)


retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5}), llm=llm
)


# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Realizar una consulta
question = "que es la presencia Yo Soy?"
unique_docs = retriever_from_llm.invoke(question)
unique_docs


INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different versions of the original question:', 'Que es la presencia de "Yo Soy" en contextos culturales y sociales?', '¿Qué características o elementos definen la presencia de "Yo Soy" en diferentes ámbitos, como la literatura, el arte o la política?', '¿Cómo se relaciona la presencia de "Yo Soy" con conceptos filosóficos como la identidad, la espiritualidad o la conciencia en textos y discursos variados?']


[Document(metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobado a través de muchas centurias estas instru

In [4]:
from langchain.schema.runnable import RunnableLambda

def remove_duplicate_documents(documents):
    """Elimina documentos duplicados basados en page_content."""
    seen_content = set()
    unique_documents = []
    for doc in documents:
        if doc.page_content not in seen_content:
            seen_content.add(doc.page_content)
            unique_documents.append(doc)
    return unique_documents


# Crear un RunnableLambda a partir de la función remove_duplicate_documents
remove_duplicates_runnable = RunnableLambda(lambda docs: remove_duplicate_documents(docs))


In [46]:
remove_duplicate_documents(unique_docs)

[Document(id='5f6b9134-460b-474d-bbc6-0604b0a495cb', metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobad

In [39]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)

In [47]:
cadena = retriever_from_llm | remove_duplicates_runnable

In [48]:
cadena.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different versions of the original question:', 'Que es la presencia de "Yo Soy" en el contexto cultural y social latinoamericano?', '¿Qué características y connotaciones tiene la frase "Yo Soy" en diferentes países y comunidades de América Latina?', '¿Cómo se relaciona la idea de identidad y pertenencia que subyace a la expresión "Yo Soy" con la experiencia histórica y política de los pueblos latinoamericanos?']


[Document(id='ee8547b9-a2d9-45c6-8923-d7ccf634b996', metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobad

In [27]:
# Ya teniendo el runable si podemos crear la funcion que retorne la cadena runable

from langchain_pinecone import PineconeVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain.schema.runnable import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import ChatOllama
import os

PINECONE_API_KEY="d078b458-2829-4a0e-adb8-534011e5d430"
os.environ["PINECONE_API_KEY"] =  PINECONE_API_KEY



def remove_duplicate_documents(documents):
    """Elimina documentos duplicados basados en page_content."""
    seen_content = set()
    unique_documents = []
    for doc in documents:
        if doc.page_content not in seen_content:
            seen_content.add(doc.page_content)
            unique_documents.append(doc)
    return unique_documents



def retriever_multi_query():

    index_name = "lanto"

    embed = OllamaEmbeddings(model="mxbai-embed-large")
    vector_store = PineconeVectorStore(embedding=embed, index_name = index_name)

    llm = ChatOllama(model="llama3.1",temperature=0.5)


    retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5}), llm=llm)
    remove_duplicates_runnable = RunnableLambda(lambda docs: remove_duplicate_documents(docs))

    cadena = retriever_from_llm | remove_duplicates_runnable 

    return cadena 

In [28]:
pregunta = "que es la presencia yo soy"
documentos = retriever_multi_query().invoke(pregunta)
documentos



[Document(metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobado a través de muchas centurias estas instru

In [29]:
# toca hacer una version 2 para pasarla al grader documentos, es decir a la funcion que califica si los documentos son relevantes 

In [30]:

def retriever_multi_query2(query):

    index_name = "lanto"

    embed = OllamaEmbeddings(model="mxbai-embed-large")
    vector_store = PineconeVectorStore(embedding=embed, index_name = index_name)

    llm = ChatOllama(model="llama3.1",temperature=0.5)


    retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5}), llm=llm)
    remove_duplicates_runnable = RunnableLambda(lambda docs: remove_duplicate_documents(docs))

    cadena = retriever_from_llm | remove_duplicates_runnable | RunnableLambda(lambda docs: {"docs": docs, "query": query})

    return cadena 

In [31]:
retriever_multi_query2(pregunta).invoke(pregunta)

{'docs': [Document(metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobado a través de muchas centurias est

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub


# Prompt
#grade_prompt = hub.pull("efriis/self-rag-retrieval-grader")

prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un evaluador que determina la relevancia de un documento recuperado respecto a una pregunta del usuario. No se requiere una prueba estricta. El objetivo es filtrar recuperaciones erróneas. Si el documento contiene palabras clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante.

Proporciona una puntuación binaria ('0' o '1') para indicar si el documento es relevante para la pregunta."""),
    ("human", "Documento recuperado: {document}\nPregunta del usuario: {question}"),
])

llm = ChatOllama(model="llama3.1",temperature=0.0)

cadena2 = prompt | llm 

filtro = cadena2.invoke({"question": "el documento es relevante", "document": "este documento es relevante"})

valor = filtro

print(valor)

if "1" in valor.content:
    print("genial es relevante")



content='La respuesta es: 1' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2024-10-03T12:54:15.38396347Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2037109039, 'load_duration': 1741355345, 'prompt_eval_count': 138, 'prompt_eval_duration': 66769000, 'eval_count': 7, 'eval_duration': 103844000} id='run-2d5be3d1-8fc3-4340-873a-3f41b9433f32-0' usage_metadata={'input_tokens': 138, 'output_tokens': 7, 'total_tokens': 145}
genial es relevante


In [12]:
def filtrado_relevante(docs, query):
    
    doc_filtrados = []
   
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un evaluador que determina la relevancia de un documento recuperado respecto a una pregunta del usuario. No se requiere una prueba estricta. El objetivo es filtrar recuperaciones erróneas. Si el documento contiene palabras clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante.

        Proporciona una puntuación binaria ('0' o '1') para indicar si el documento es relevante para la pregunta."""),
        ("human", "Documento recuperado: {document}\nPregunta del usuario: {question}"),
        ])
   
    llm = ChatOllama(model="llama3.1",temperature=0.0)

    cadena2 = prompt | llm 

    for doc in docs:
    
      filtro = cadena2.invoke({"question": query, "document": doc})

      if "1" in filtro.content:
        doc_filtrados.append(doc)

    return doc_filtrados
      


filtrado_relevante(documentos, pregunta)

[Document(metadata={'page': 31.0, 'source': 'El libro de oro.pdf'}, page_content='muchísimo su propia evolución y bajo ningún aspecto es admisible. \nCada estudiante debe comprender que su única incumbenc ia es la de armonizar, apurar y expandir su \npropia mente y su mundo. Una vez que los estudiant es comprendan que la única demanda imperativa de la \n«Gran Ley de su Ser»  es la armonía de su mente y sentimientos , la Perfección se manifestará rápidamente. \nSi esto no es mantenido, no podrán pasar de cierto grado de progreso. \nTan pronto como los estudiantes se den cuenta de es to, y comiencen a usar la Presencia «YO SOY» \nordenando la armonía y el silencio de su actividad exte rna se darán cuenta que podrán ver, sentir y ser la \nPerfección que han deseado tanto. Cuando los estudiantes  y amigos tienen un profundo y sincero Amor para \ncada cual, ese Amor es la más grande bendición y el poder más estimulante. Esta es una manera de que el \nestudiante se examine cons tantemente, p

In [17]:
pregunta = "que es la presencia yo soy"
documentos = retriever_multi_query().invoke(pregunta)
documentos

[Document(metadata={'page': 4.0, 'source': 'El libro de oro.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobado a través de muchas centurias estas instru

In [18]:

documentos_relevantes = filtrado_relevante(documentos, pregunta)

documentos_relevantes

[Document(metadata={'page': 28.0, 'source': 'El libro de oro.pdf'}, page_content='toda ocasión, entonces sí sabes que tienes la Presencia dentro de ti. \nEl próximo paso es determinar: «YO SOY la Presencia iluminadora, por  la cual nada que yo necesite saber \npuede ser sustraído, ya que "YO SOY" la Sabiduría, "Y O SOY" el Poder revelador que me trae todo ante mí \npara yo poder comprender y actuar de acuerdo».  \nEs muy fácil una vez que se ha comprendido que «YO SOY la Única Inteligencia y la Única Presencia \nactuando». el ver cómo tienes tú el cetro entre tus manos físicas y a través de es ta Presencia «YO SOY» \npuedes obligar a que todo lo que tú necesites saber te  sea revelado. Y yo te aseguro que esto en ninguna \nforma interfiere con el libre albedrío de ningún otro  individuo y que no hay error ni ningún daño en reclamar y \npedir lo que es de uno propio, pues al hacer esto no se está interfiriendo con nadie. \nSi en cualquier momento alguien hace por quitamos lo que nos per

In [34]:
pregunta = "que es la presencia yo soy"
docs_query = retriever_multi_query2(pregunta).invoke(pregunta)
filtrado_relevante(docs_query["docs"], docs_query["query"])



[Document(metadata={'page': 28.0, 'source': 'El libro de oro.pdf'}, page_content='toda ocasión, entonces sí sabes que tienes la Presencia dentro de ti. \nEl próximo paso es determinar: «YO SOY la Presencia iluminadora, por  la cual nada que yo necesite saber \npuede ser sustraído, ya que "YO SOY" la Sabiduría, "Y O SOY" el Poder revelador que me trae todo ante mí \npara yo poder comprender y actuar de acuerdo».  \nEs muy fácil una vez que se ha comprendido que «YO SOY la Única Inteligencia y la Única Presencia \nactuando». el ver cómo tienes tú el cetro entre tus manos físicas y a través de es ta Presencia «YO SOY» \npuedes obligar a que todo lo que tú necesites saber te  sea revelado. Y yo te aseguro que esto en ninguna \nforma interfiere con el libre albedrío de ningún otro  individuo y que no hay error ni ningún daño en reclamar y \npedir lo que es de uno propio, pues al hacer esto no se está interfiriendo con nadie. \nSi en cualquier momento alguien hace por quitamos lo que nos per

In [42]:
# aqui toca modificar el runable para que reciba el diccionario que da el multi query
remove_no_relevant_docs_runnable = RunnableLambda(lambda diccionario: filtrado_relevante(diccionario["docs"],diccionario["query"]))

In [44]:
cadena3 = retriever_multi_query2(pregunta) | remove_no_relevant_docs_runnable
cadena3.invoke(pregunta)

[Document(metadata={'page': 28.0, 'source': 'El libro de oro.pdf'}, page_content='toda ocasión, entonces sí sabes que tienes la Presencia dentro de ti. \nEl próximo paso es determinar: «YO SOY la Presencia iluminadora, por  la cual nada que yo necesite saber \npuede ser sustraído, ya que "YO SOY" la Sabiduría, "Y O SOY" el Poder revelador que me trae todo ante mí \npara yo poder comprender y actuar de acuerdo».  \nEs muy fácil una vez que se ha comprendido que «YO SOY la Única Inteligencia y la Única Presencia \nactuando». el ver cómo tienes tú el cetro entre tus manos físicas y a través de es ta Presencia «YO SOY» \npuedes obligar a que todo lo que tú necesites saber te  sea revelado. Y yo te aseguro que esto en ninguna \nforma interfiere con el libre albedrío de ningún otro  individuo y que no hay error ni ningún daño en reclamar y \npedir lo que es de uno propio, pues al hacer esto no se está interfiriendo con nadie. \nSi en cualquier momento alguien hace por quitamos lo que nos per

In [46]:
# Contruyamos la funcion ahora

def retriever_filtrado_final(query):

    remove_no_relevant_docs_runnable = RunnableLambda(lambda diccionario: filtrado_relevante(diccionario["docs"],diccionario["query"]))
    return retriever_multi_query2(pregunta) | remove_no_relevant_docs_runnable






In [47]:
cadena_retriever_final = retriever_filtrado_final(pregunta)
cadena_retriever_final.invoke(pregunta)

[Document(metadata={'page': 28.0, 'source': 'El libro de oro.pdf'}, page_content='toda ocasión, entonces sí sabes que tienes la Presencia dentro de ti. \nEl próximo paso es determinar: «YO SOY la Presencia iluminadora, por  la cual nada que yo necesite saber \npuede ser sustraído, ya que "YO SOY" la Sabiduría, "Y O SOY" el Poder revelador que me trae todo ante mí \npara yo poder comprender y actuar de acuerdo».  \nEs muy fácil una vez que se ha comprendido que «YO SOY la Única Inteligencia y la Única Presencia \nactuando». el ver cómo tienes tú el cetro entre tus manos físicas y a través de es ta Presencia «YO SOY» \npuedes obligar a que todo lo que tú necesites saber te  sea revelado. Y yo te aseguro que esto en ninguna \nforma interfiere con el libre albedrío de ningún otro  individuo y que no hay error ni ningún daño en reclamar y \npedir lo que es de uno propio, pues al hacer esto no se está interfiriendo con nadie. \nSi en cualquier momento alguien hace por quitamos lo que nos per

# Probando las funciones desdee los modulos

In [1]:
from carga_libros import cargar_libros

In [2]:
ruta = "El libro de oro.pdf"
comentario = "este libro es genial y trata de tu potencial espiritual"
titulo = "otro"
cargar_libros(ruta,titulo, comentario)

'El documento otro ha sido cargado con éxito'

In [4]:
cargar_libros(ruta,titulo, comentario)

'Ya existe este documento en la base de datos vectorial'

In [3]:
from funcion_retrievel import  retriever_multi_query, retriever_filtrado_final


In [6]:

pregunta = "que es la presencia yo soy"

resultado = retriever_multi_query(pregunta).invoke(pregunta)
resultado 

{'docs': [Document(metadata={'page': 4.0, 'source': '/tmp/tmpscfy2kq6.pdf'}, page_content='5determinando cómo quieres que actúen para ti. \nNadie puede preguntar: ¿Y cómo es que yo hago para calificar la energía? Todo el mundo conoce la \ndiferencia entre lo destructi vo y lo constructivo en pensam iento, sentimiento y acción. \nEl estudiante, al recibir esta in strucción, debe constantemente analizar el  motivo que lo impele para detectar \nsi hay algún sentimiento de orgullo intelectual, de arroganc ia o de testarudez en la mente y cuerpo exterior. Si \nhay algún deseo solapado de discutir o de probar que la in strucción está errada, en lugar de recibir la \nBendición y la Verdad, el individuo ha cerrado incon scientemente la puerta, y por el momento ha anulado su \nhabilidad de recibir el bien ofrecido. \nTambién quiero recordarles a los discípulos, que no obs tante sus opiniones personales respecto a lo que \ndebe ser o no la Verdad, yo he comprobado a través de muchas centurias es

In [7]:
pregunta = "que es la presencia yo soy"
resultado_final = retriever_filtrado_final(pregunta)
resultado_final.invoke(pregunta)

[Document(metadata={'page': 28.0, 'source': 'El libro de oro.pdf'}, page_content='toda ocasión, entonces sí sabes que tienes la Presencia dentro de ti. \nEl próximo paso es determinar: «YO SOY la Presencia iluminadora, por  la cual nada que yo necesite saber \npuede ser sustraído, ya que "YO SOY" la Sabiduría, "Y O SOY" el Poder revelador que me trae todo ante mí \npara yo poder comprender y actuar de acuerdo».  \nEs muy fácil una vez que se ha comprendido que «YO SOY la Única Inteligencia y la Única Presencia \nactuando». el ver cómo tienes tú el cetro entre tus manos físicas y a través de es ta Presencia «YO SOY» \npuedes obligar a que todo lo que tú necesites saber te  sea revelado. Y yo te aseguro que esto en ninguna \nforma interfiere con el libre albedrío de ningún otro  individuo y que no hay error ni ningún daño en reclamar y \npedir lo que es de uno propio, pues al hacer esto no se está interfiriendo con nadie. \nSi en cualquier momento alguien hace por quitamos lo que nos per

In [ ]:
# funionan correctamente, ahora |vemoas la cadena completa del RAG

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from funcion_retrievel import  retriever_multi_query, retriever_filtrado_final

prompt = ChatPromptTemplate.from_messages([
  ("human", """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use 8 sentences maximum and keep the answer concise and ensure it contains relevant information.
Question: {question} 
Context: {context} 
Answer:"""),
])

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)


In [5]:
pregunta = "que es la presencia yo soy"
resultado_final = retriever_filtrado_final(pregunta)
docs = resultado_final.invoke(pregunta)


rag_chain = prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"context": docs, "question": pregunta})

In [18]:
generation

'La Presencia "YO SOY" es la presencia iluminadora y sabiduría que actúa dentro de cada ser humano. Es el poder revelador que trae todo ante uno para comprender y actuar de acuerdo. También se refiere a la invencible Presencia de Dios, que es la actividad inteligente del mundo y sus asuntos.'

In [7]:
docs

[Document(metadata={'page': 46.0, 'source': 'El libro de oro.pdf'}, page_content='mismos, de la calmada intensidad equilibrada por la cual se vuelven conscientes de su «Presencia YO SOY».  \nTe traigo estas grandes nuevas, porque yo la s comprobé en mi experiencia personal. \nAntes de que decidiera completamente la manera en que Yo daría el ejemplo a la humanidad, comencé de \nrepente a usar la afirmación que me vino de un impulso interno: «YO SOY la Resurrección y la Vida».  Dos días \ndespués que comencé a usar esa afirmación con gran r egocijo, vi lo que se debía hacer, y deseo asegurarte \nque fue el uso consciente de la Poderosa Afirmación: «YO SOY la Resurrección y la Vida»,  que me dio el \npoder para hacer la ascensión en presencia de tantos y r egistrar en los archivos etéricos ese ejemplo que se \nmantendrá presente eternament e para toda la humanidad. \nDesafortunadamente, el velo de la idea ortodoxa cubrió  las mentes de la gente, impidiendo la comprensión'),
 Document(meta

In [26]:
def referencias_docs(docs):
    
    """
    Genera una cadena de referencias a partir de una lista de documentos.

    Args:
        docs (list): Lista de documentos, donde cada documento es un objeto con un atributo `metadata` que contiene información sobre la página y el origen.

    Returns:
        str: Cadena de referencias en formato "Título [fuente] y Página [página]", separadas por comas y espacios.

    Ejemplo:
        >>> docs = [doc1, doc2, doc3]
        >>> referencias = referencias_docs(docs)
        >>> print(referencias)
        Título El libro de oro.pdf y Página 28, Título El libro de oro.pdf y Página 71, Título El libro de oro.pdf y Página 32, Título El libro de oro.pdf y Página 1
    """
    
    referencia = []
    for doc in docs:
        referencia.append(f"Título {doc.metadata['source']} y Página {doc.metadata['page']}")

    referencia = ", ".join(referencia)

    return referencia

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from funcion_retrievel import  retriever_multi_query, retriever_filtrado_final



def generation_retriever(query):
    
    prompt = ChatPromptTemplate.from_messages([
    ("human", """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use 8 sentences maximum and keep the answer concise and ensure it contains relevant information.
    Question: {question} 
    Context: {context} 
    Answer:"""),
    ])
    llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    
    )
    
    resultado_final = retriever_filtrado_final(pregunta)
    docs = resultado_final.invoke(pregunta)

    referencias = referencias_docs(docs)  
    rag_chain = prompt | llm | StrOutputParser()
    generation = rag_chain.invoke({"context": docs, "question": query})

    
    
    return generation + referencias






In [29]:
pregunta = "que es la presencia yo soy"
print(generation_retriever(pregunta))

La "Presencia YO SOY" se refiere a la conciencia divina y omnipotente que reside en cada individuo. Es la fuente de la Eterna Juventud y Belleza, la Resurrección y la Vida, y el poderoso motor que impulsa la acción y la expresión humana. La "Presencia YO SOY" es la verdad fundamental y la realidad última de cada ser humano, y su reconocimiento y aceptación pueden llevar a una comprensión profunda de la vida y la manifestación de la realidad deseada.Título El libro de oro.pdf y Página 28.0, Título El libro de oro.pdf y Página 32.0, Título El libro de oro.pdf y Página 71.0, Título /tmp/tmpscfy2kq6.pdf y Página 41.0, Título El libro de oro.pdf y Página 46.0, Título El libro de oro.pdf y Página 4.0
